In [426]:
#importing library
from bs4 import BeautifulSoup ,NavigableString, Comment
import requests
import time
import re
from datetime import datetime
def get_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,  'html.parser')
    article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
    #remove the last 4 element
    article.find('p', class_ = 'tuht_all').decompose()
    article.find('div',class_  = 'bv-lq').decompose()
    article.find('div', id = 'zone_banner_sponser_product').decompose()
    article.find('p', class_ = 'linkOrigin').decompose()
    
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    # set attribut for img
    b = article.find_all('img')
    for i in b:
        if '.svg' in i['src']:
            i.decompose()
    for i in article.find_all('img'):
        if 'data-original' in i.attrs.keys():
            i['src'] = i.get('data-original')
        #i.attrs = ['class', 'alt', 'src', 'data-original']
        del i['onclick']
        del i['style']
        del i['class']
    #remove js  and css
    h2 = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find('h2')
    h2_text = '\n'.join(line.strip() for line in h2.get_text().split('\n') if line.strip())
    # Update the text of the h2 tag
    h2.string = h2_text
    #soup2 = BeautifulSoup("", 'html.parser')
    #soup2.append(h2)
    for script_or_style in article(['script', 'style']):
        script_or_style.decompose()
    for i in article.find_all(recursive = True):
        if i.name != 'p':
            try:
                del i['onclick']
                del i['id']
                del i['class']
                del i['style']
            except AttributeError:
                continue
            except TypeError:
                continue
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        i['width'] = 800
        i['height'] = 400
    for video in article.find_all('video'):
        video.decompose()
    for i in article.find_all('p', class_ = 'img_chu_thich_0407'):
            i['style'] = "text-align: center;"
            del i['class']
    for i in article.find_all('div', {'align': 'center'}):
        i.decompose()
    for element in article.find_all(recursive = True,string=True):
        if isinstance(element, NavigableString) and element.strip() == '':
            element.extract()
    for i in article.find_all('div'):
        if i.children == None and i.string == None:
            i.decompose()
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: 24h.com.vn"  # Set the content of <i> tag
    # Append the <i> tag as the last child of the <article> tag
    article.append(source_tag)
    return article
#print(len(caption_text_list))

#convert time from post to the format of output
def convert_string(input_str):
    # Extract the date part using regular expression
    pattern = r'\d{2}/\d{2}/\d{4}'
    date_part = re.search(pattern, input_str).group()
    match = re.search(pattern, input_str)
    if match:
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_part, '%d/%m/%Y')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    else:
        return ""
#Convert time from post to datetime format
def convert_time_string(posted_date):
    pattern = r'\d{2}/\d{2}/\d{4}'
    match = re.search(pattern, posted_date)

    if match:
        date_string = match.group()

        # Convert to datetime object
        datetime_obj = datetime.strptime(date_string, "%d/%m/%Y")
        return datetime_obj
    else:
        return ''
def get_post(url):
    try:
        response = requests.get(url)
        time.sleep(3)
        soup = BeautifulSoup(response.content, 'html5lib')
        content = get_content(url)
        post_time = soup.find('time').text.strip()
        published_date = convert_string(post_time)
        title = soup.find('h1').text.strip()
        #h2 = soup.find('h2').text.strip()
        #images_src = [i.attrs['src'] for i in soup.find('article', class_= 'cate-24h-foot-arti-deta-info').find_all('img')[:-1] if 'svg' not in i.attrs['src']]
        #images_src = [img['data-original'] if 'https://image-us.24h.com.vn' not in img['src'] else img['src'] if 'svg' not in img['src'] else '' for img in soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find_all('img')[:-1]]
        #text_list = [ child.text for child in soup.find('article', class_= 'cate-24h-foot-arti-deta-info').find_all('p')[:-3] if re.sub(r'\n+', '', child.text) != ""]
        #text_list = [h2] + text_list
        return content,title,published_date
    except AttributeError as e:
        print(e)
        content = ''
        title = ''
        published_date= ''
        return content,title,published_date

def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time()-3*24*3600)
    for i in urls:
        response = requests.get(i)
        soup = BeautifulSoup(response.content, 'html5lib')
        try:
            date_posted = soup.find('time').text.strip()
            date_posted_norm = convert_time_string(date_posted)
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(i)
                print(i)
        except AttributeError as e:
            print(e)
            continue
    return filtered_urls
def get_list_url(cat_url):
    urls = []
    response = requests.get(cat_url)
    time.sleep(3)
    soup = BeautifulSoup(response.content,'html5lib')
    news_block  = soup.find('section', id = 'tin_bai_noi_bat_khac')
    try:
        url_list = news_block.find_all('a')
        print(len(url_list))
        for url in url_list:
            if 'https://www.24h.com.vn/' in url['href']:
                if url['href'] not in urls:
                    urls.append(url['href'])
    except AttributeError as e:
        print(e)
    return urls
def add_list(web_24h_com_vn):
    for i in list(web_24h_com_vn['urls'].keys()):
        for j in list(web_24h_com_vn['urls'][i]['sub-category'].keys()):
            
            urls = get_list_url(web_24h_com_vn['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_24h_com_vn['urls'][i]['sub-category'][j]['url'])
            web_24h_com_vn['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)

def add_post(web_24h_com_vn):
    for i in list(web_24h_com_vn['urls'].keys()):
        for j in list(web_24h_com_vn['urls'][i]['sub-category'].keys()):
            web_24h_com_vn['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_24h_com_vn['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['title'],web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_24h_com_vn['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_24h_com_vn['urls'][i]['cate_id'],web_24h_com_vn['urls'][i]['sub-category'][j]['name'],web_24h_com_vn['urls'][i]['sub-category'][j]['name'],web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['title'],web_24h_com_vn['urls'][i]['sub-category'][j]['url_list'][u])
                


def get_news():
    web_24h_com_vn = {
        "home_page":"https://www.24h.com.vn/",
        "urls":{
            "youths":
            {
            "url":"https://www.24h.com.vn/ban-tre-cuoc-song-c64.html",
            "cate_id":60,
             "sub-category":{
                0:{"name":"Chuyện công sở","url":"https://www.24h.com.vn/chuyen-cong-so-c180.html"},
                1:{"name":"Tình yêu - Giới Tính","url":"https://www.24h.com.vn/tinh-yeu-gioi-tinh-c306.html"},
                2:{"name":"Ngoại tình","url":"https://www.24h.com.vn/ngoai-tinh-c435.html"},
                3:{"name":"Giới trẻ","url":"https://www.24h.com.vn/gioi-tre-c434.html"},
                4:{"name":"Hotgirl - Hotboy","url":"https://www.24h.com.vn/hotgirl-hot-boy-c64e3398.html"},
                5:{"name":"Nhịp sống trẻ","url":"https://www.24h.com.vn/nhip-song-tre-c685.html"}
             }
            }
  
        }
    }
#
    add_list(web_24h_com_vn)
    add_post(web_24h_com_vn)
    return web_24h_com_vn
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "token": "draftpost",#"5goalvodichcmnl",  # Replace with your actual access token
        "published_date": published_date,
        "category_id": category_id , # Replace with the actual category ID as required
        "domain":"24h"
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    web_24h_com_vn = get_news()
    for i in list(web_24h_com_vn['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(web_24h_com_vn['urls'][i]['sub-category']):
            url_list =  web_24h_com_vn['urls'][i]['sub-category'][j]['url_list']
            for t in range(0,len(url_list)):
                text = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['text']
                title = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = web_24h_com_vn['urls'][i]['cate_id']
                #send_post_to_5goals(title,text,cate_id,published_date)
                print(i,j,titlepublished_date)


In [235]:
web_24h_com_vn = get_news()

26
youths 0 https://www.24h.com.vn/chuyen-cong-so-c180.html
'NoneType' object has no attribute 'text'
29
youths 1 https://www.24h.com.vn/tinh-yeu-gioi-tinh-c306.html
'NoneType' object has no attribute 'text'
https://www.24h.com.vn/ban-tre-cuoc-song/vo-doi-ly-hon-lien-tuc-toi-phai-lam-sao-c64a1516478.html
https://www.24h.com.vn/ban-tre-cuoc-song/ban-trai-doi-nguoi-yeu-tra-tien-thue-nha-khi-cung-song-trong-can-ho-cua-anh-ay-luong-tang-thi-tien-thue-cung-tang-theo-c64a1523719.html
29
youths 2 https://www.24h.com.vn/ngoai-tinh-c435.html
29
youths 3 https://www.24h.com.vn/gioi-tre-c434.html
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find_all'
youths 4 https://www.24h.com.vn/hotgirl-hot-boy-c64e3398.html
29
youths 5 https://www.24h.com.vn/nhip-song-tre-c685.html
youths 1 60 Tình yêu - Giới Tính Tình yêu - Giới Tính Vợ đòi ly hôn liên tục, tôi phải làm sao? https://www.24h.com.vn/ban-tre-cuoc-song/vo-doi-ly-hon-lien-tuc-toi-phai-lam-sao-c64a1516478.html
yout

In [427]:
get_content('https://www.24h.com.vn/doi-song-showbiz/tran-quan-hy-lan-dau-noi-ve-1400-anh-nong-hai-doi-truong-ba-chi-chung-han-dong-c729a1521545.html')

<article class="cate-24h-foot-arti-deta-info"><h2>Đây là lần hiếm hoi sao nam nói thẳng về scandal tai tiếng 15 năm trước.</h2><p>Trần Quán Hy từng là tài tử hàng đầu làng giải trí xứ Hương Cảng. Vào thời kỳ đỉnh cao, anh có vẻ ngoài điển trai, lãng tử cùng lối diễn xuất đa dạng, được hàng triệu cô gái ở châu Á ngưỡng mộ.</p><p>Vào năm 2008, một vụ scandal đã làm chấn động làng giải trí xứ Trung khi hàng nghìn bức ảnh ăn chơi thác loạn của Trần Quán Hy cùng nhiều cô gái nóng bỏng bị phát tán ra ngoài. Điều đáng nói là những ảnh giường chiếu này còn có sự xuất hiện của các mỹ nhân nổi tiếng của làng giải trí Hoa ngữ thời bấy giờ như: Trương Bá Chi, Chung Hân Đồng, Maggie Q, Trần Văn Viên…</p><p><div><div><div><section><div></div></section><img alt='Trần Quán Hy lần đầu nói về 1.400 ảnh nóng "hại đời" Trương Bá Chi, Chung Hân Đồng - 1' class="aligncenter" height="400" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-24/96a0b513be3b435d8a0145e428be6f0a-1700806904-653-width108

In [215]:
main()

KeyboardInterrupt: 

In [214]:
article['class']

['cate-24h-foot-arti-deta-info']

In [136]:
for i in caption_text_list:
    print(i)

Cô dâu Thanh Hóa đeo chi chít vàng trong ngày cưới
Cô dâu gây chú ý bởi nhan sắc xinh đẹp
Cặp đôi có 5 năm bên nhau trước khi về chung một nhà
Cặp đôi từng trải qua biến cố lớn trong hôn nhân

Vợ chồng Quỳnh Như đã "gương vỡ lại lành"


Cô dâu Thanh Hóa ngày càng xinh đẹp.


[caption id="" align="aligncenter" width="800"]Cô dâu Thanh Hóa đeo chi chít vàng trong ngày cưới[/caption]
[caption id="" align="aligncenter" width="800"]Cô dâu gây chú ý bởi nhan sắc xinh đẹp[/caption]
[caption id="" align="aligncenter" width="800"]Cặp đôi có 5 năm bên nhau trước khi về chung một nhà[/caption]
[caption id="" align="aligncenter" width="800"]Cặp đôi từng trải qua biến cố lớn trong hôn nhân[/caption]
[caption id="" align="aligncenter" width="800"][/caption]
[caption id="" align="aligncenter" width="800"]Vợ chồng Quỳnh Như đã "gương vỡ lại lành"[/caption]
[caption id="" align="aligncenter" width="800"][/caption]


In [355]:
response = requests.get('https://www.24h.com.vn/ban-tre-cuoc-song/cuoc-song-it-ai-biet-cua-co-be-xinh-dep-tung-khien-ca-the-gioi-kinh-ngac-vi-qua-thong-minh-c64a1524455.html')
soup = BeautifulSoup(response.content,  'html.parser')
#remove the last 4 element
article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
for script_or_style in article(['script', 'style']):
    script_or_style.decompose()

In [408]:
caps = []
for i in article.find_all('img')[0:1]:
    test = i.parent
    next_sibling_of_parent = test.find_next_sibling('p', class_ = 'img_chu_thich_0407' )
    # Optionally, skip non-tag elements such as NavigableString (like whitespace):
    while next_sibling_of_parent and isinstance(next_sibling_of_parent, str):
        next_sibling_of_parent = next_sibling_of_parent.find_next_sibling()
        caps.append(next_sibling_of_parent)
    

In [409]:
article.find_all('img')[0].parent.next_sibling.next_sibling.next_sibling

In [410]:
caps

[]

In [368]:
for i in article.find_all('p', class_='img_chu_thich_0407'):
    print(i.previous_sibling)

In [369]:
article

<article class="cate-24h-foot-arti-deta-info">
<h2 class="cate-24h-foot-arti-deta-sum ctTp tuht_show" id="article_sapo">
<strong>
                    Thoạt nhìn, Isla có vẻ giống một đứa trẻ 2 tuổi, nhưng cô bé thông minh nhất thế giới này có thể làm được những điều mà không ai tưởng tượng nổi.                </strong>
</h2>
 <p>Một cô bé sống ở Mỹ trở thành thành viên nhỏ nhất người Mỹ gia nhập hội Mensa High IQ – một tổ chức dành cho những người thông minh nhất thế giới khi mới 2 tuổi thu hút sự chú ý của nhiều người.</p>
<p>Cô bé "thông minh nhất thế giới" có tên là Isla McNabb. Cha mẹ của cô bé là Amanda và Jason McNabb. Cha mẹ cho biết cô bé rất thông minh và dấu hiệu ngày càng rõ ràng khi cô bắt đầu học. Mới 1 tuổi, Isla đã bắt đầu học màu sắc, số và bảng chữ cái.</p>
<p>Jason McNabb, cha của Isla cho biết: "Khi được 18 tháng, cô bé đã tự học bảng chữ cái và bắt đầu biết đọc khi mới 2 tuổi".</p>
<p align="center"><div id="container-24h-banner-in-image" style="text-align: center;w

In [387]:
for i in article.find_all('p', align = "center", recursive = True):
    print(i.find('img').parent)

<div style="position:relative;max-width: 740px;margin: 0 auto">
<img alt="Cuộc sống ít ai biết của cô bé xinh đẹp, từng khiến cả thế giới kinh ngạc vì quá thông minh - 1" class="news-image" onclick="show_slide_image_news(0)" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-12-05//1701757221-cuoc-song-hien-tai-cua-co-be-thong-minh-nhat-the-gioi-ai-cung-tram-tro-width900height672.jpg" width="740"/>
<div id="24h-banner-in-image" style="left: 50%;transform: translateX(-50%);position: absolute;display: block;height: 90px;color: white;bottom: 0px">
<div class="txtCent" id="ixhFnVosny_139_15s">
<div class="text_adver_right" onclick="in_image_close();"></div>

</div>
<div id="24h-banner-in-image-close" style="top:2px;right: 0px;position: absolute;display: none;overflow: hidden;width: 22px;height: 22px;color: #ccc;font-size: large;">
<img class="img_tin_lien_quan_trong_bai" id="in-image-close" onclick="in_image_close();" src="https://image-us.24h.com.vn/upload/24h_banner_image_library